## Integrantes:
1. Camila Coltriani
2. Luis Dartayet
3. Irania Fuentes
4. Jonathan Fichelson
5. Ornella Cevoli
# Trabajo práctico 2 : Modelo de regresión lineal del dataset Properatti
## Definimos las variable objetivo y predictoras


En el TP1 fue planteada la hipótesis que el precio (variable objetivo) de las propiedades iba a estar influenciado principalmente por la superficie y la ubicación (variables predictoras).

Por lo cual, en este trabajo, **el objetivo es predecir el valor en dolares de las propiedades** utilizando diferentes variables de superficie y ubicación.

In [ ]:
#Las librerías utilizadas en este documento son:
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn import metrics
from matplotlib.ticker import ScalarFormatter
from matplotlib import gridspec
sns.set()

## Visualización del dataset

In [ ]:
# Visualización de la forma y atributos del dataset limpio del TP1
data = pd.read_csv("./data/data_limpio_gdf.csv")
pd.options.display.max_columns
print(data.shape)
print("El dataset está compuesto por:", data.shape[0], "filas y",data.shape[1],"columnas.")
data.sample(5)

### Descripción de las columnas del dataset limpio:

Las columnas que incluye son:

● municipio: ubicación del inmueble por su municipio/barrios

● provincia: ubicación del inmueble por provincia

● lat  ●lon: ubicación de latitud y longitud

● superficie_m2_total: superficie total en m² del inmueble

● price_usd: Precio en dólares del inmueble

● tipo: tipo de inmueble en venta (casa, departamento, ph, tienda)

● ambientes_cat: cantidad de ambientes del inmueble (0, 1, 2, 3 , 4 o más)

● precio_usd_por_m2: Precio en dólares por metro cuadrado (USD/m²: precio dólares / superficie)

● tipo_cat_code: categoría numérica de tipo de inmueble

● municipio_cat_code: categoría numérica de municipios

● provincia_cat_code: categoría numérica de provincia

● tipo_cat_code: categoría numérica de ambientes_cat

● geometry: figura geométrica de latitud y la longitud

● country_name: nombre del país donde ocurre la operación inmobiliaria

● precio_usd_por_m2_cat: categoría numérica de precio_usd_por_m2

# Revision de datos nulos

In [ ]:
missing_by_row=data.isna().sum().sort_values(ascending=False)[0:6]
missing_by_row
#Revisamos la presencia de datos NaN
#La columna "ambientes_cat" tiene con 1248 registros nulos

- Reviso donde están ubicados y a que propiedad pertenecen los registros nulos para saber si afectaran escoger EL tipo de inmueble y su zona

In [ ]:
#reviso donde están ubicados y a que propiedad pertenecen los registros nulos para saber si afectaran escoger un tipo de inmueble y su zona
mascara_nulos = data["ambientes_cat"].astype(str) == "nan" 
data_nulos = data[mascara_nulos]
data_nulos.loc[:, ["municipio", 'tipo', 'ambientes_cat', "precio_usd"]].sample(7)
#print(data[mascara_nulos].index)

In [ ]:
#se realiza lo siguiente solo a fines de graficar en presentacion.
missing_by_row_2= data_nulos.groupby('municipio')['tipo'].count().sort_values(ascending=False).head(25)
missing_by_row_2_porc= missing_by_row_2/data['ambientes_cat'].isna().sum()*100
pareto=missing_by_row_2_porc.values
acum=[]
val_acum=0
for i in missing_by_row_2_porc:
    val_acum= val_acum+i
    acum.append(val_acum)
pareto=acum
pareto
# print(data['municipio'].unique().shape)

# #Revisamos la distribución de los nulos por municipio
fig=plt.figure()
ax= fig.add_subplot(1,1,1)
ax.set_title('Pareto Municipios')
ax.bar(missing_by_row_2.index, missing_by_row_2, color="C0")
ax2=ax.twinx()
ax2.plot(missing_by_row_2.index,pareto,color="C1",marker="D",ms=5)
# ax2.yaxis.set_major_formatter(PercentFormatter(2))
ax.tick_params(axis="y", colors="C0")
ax2.tick_params(axis="y", colors="C1")
ax.set_xticklabels(missing_by_row_2.index, rotation=90)

plt.show()



Vemos que los datos NaN están distribuidos equitativamente y no están concentrados en una mismo municipio por lo que procedemos a eliminarlos

In [ ]:
#Los elimino 
data.dropna(subset=['ambientes_cat'], inplace=True)
#print(data.isna().sum())

In [ ]:
#vemos el resumen de la tendencia central, la dispersión y la forma de la distribución de un conjunto de datos
data.describe()

Existen datos que no permiten ver los estadísticos ya que hay valores de 0 en sup_m2_total y precio_usd_por_m2: eliminarlos

In [ ]:
#eliminamos del dataset los registros de sup_m2_total con valores de cero
data.drop(data[(data["sup_m2_total"] ==0)].index, inplace=True ,axis=0)

In [ ]:
data.describe()
#Ahora si podemos ver el resumen de los estadisticos que nos serviran de referencia más adelante

## Visualización gráfica los datos para elegir el area geografica y el tipo de inmueble a modelar

In [ ]:
#graficamos las provincias y municipios que contengan un valor mínimo de 500 registros por municipio 
#consideramos que 500 es un limite de datos para que el performance del modelo
limite = 500
data = data.copy().groupby(['municipio']).filter(lambda grp: grp.shape[0] > limite)

In [ ]:
fig= plt.subplots(figsize=(20,20),constrained_layout=True)
grid = gridspec.GridSpec(2, 1, height_ratios=[1, 3])

ax1=plt.subplot(grid[0])
sns.countplot(data=data,y="provincia",order=data["provincia"].value_counts().index ,ax=ax1,color="g")

ax1.set_yticklabels(ax1.get_yticklabels(),fontsize="medium")
ax1.set_title("Distribucion de registros segun la provincia", fontsize= 'large')

ax2=plt.subplot(grid[1])
sns.countplot(data=data,x="municipio",order=data["municipio"].value_counts().index,ax=ax2,color="b")


ax2.set_title("Distribucion de registros segun los municipios", fontsize= 'large')
ax2.set_xticklabels(ax2.get_xticklabels(),rotation=90,ha="right")
plt.xticks(fontsize= 10)
plt.yticks(fontsize= 10)
ax1.grid()
ax2.grid()
plt.show()

La mayor cantidad de registros en nuestro dataset están Capital Federal para los barrios de Palermo, Belgrano, Caballito.
Consideraremos Capital Federal para la evaluación de los modelos 

In [ ]:
#Revisamos la distribución de registros por tipo de inmueble
plt.figure(figsize=(5,3))

plt.gca().yaxis.set_major_formatter(ScalarFormatter())
ax = sns.countplot(data = data, x = "tipo")
ax.set_xticklabels(ax.get_xticklabels(),rotation=40,ha="right")
plt.show()

#Apartamentos tiene la mayoría de los datos

#### Con base a que la mayor cantidad de nuestros datos se encuentra en Capitar Federal y para el tipo de inmueble Apartamentos decidimos trabajar con ellos en la implementacion de los modelos

Seleccionamos Capital Federal y apartamentos para trabajar el dataset

In [ ]:
#armamos un dataset nuevo seleccionando capital federal y apartamentos
data=data.copy()
condicion_provincia= data["provincia"]=="Capital Federal"
condicion_tipo= data["tipo"]== 'apartment'
condicion_compuesta= condicion_provincia&condicion_tipo
data = data[condicion_compuesta]
data.shape

Vemos los municipios o barrios de Capital Federal filtrados

In [ ]:
data.groupby(["provincia"])["municipio"].value_counts()

## Correlación entre la variables del dataset

In [ ]:
#analizamos la correlación entre cada una de las variables.
figz= plt.figure()
mask_cols= ["sup_m2_total","precio_usd","precio_usd_por_m2", "ambientes_cat_code","lat", "lon"]
graph=sns.pairplot(data[mask_cols])
graph.fig.set_size_inches(20,10)
plt.grid()
print("Correlacion entre variables para capital federal y apartamentos")
plt.show()


Puede verse outliers en la relacion precio_usd y sup_m2_total que podria impactar sobre las predicciones. 

Se observa en la variable Superficie para valores mayores a 800 m2 y para precios 4x1e6, los cuales eliminaremos.

Precio_usd_por_m2 es la relacion de precio en dolares y superficie total, por lo cual al utilizar precio en dolares como variable objetivo, basta con considerar la variable superficie_m2_total como predictora.

In [ ]:
#detectamos el valor maximo de superficie total
data["sup_m2_total"].max()

In [ ]:
#eliminamos el valor
data.drop(data[(data["sup_m2_total"]>=965)].index, inplace=True ,axis=0)
#corroboramos que se eliminó el outlier de superficie
#data["sup_m2_total"].max()

In [ ]:
#eliminamos outliers para valores de propiedades mayores a 4M usd
outliers_precios= data["precio_usd"]>=4000000
data.drop(data[outliers_precios].index, inplace=True ,axis=0)

In [ ]:
#Vemos la correlación numerica entre las variables 
mask_cols_corr= ["sup_m2_total","precio_usd", "ambientes_cat_code","lat", "lon"]
data_corr = data[mask_cols_corr].corr()
#graficamos
plt.figure(figsize=(6,6))
sns.heatmap(data_corr, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Correlación entre variables")
plt.show()


De este grafico podemos detectar una correlación significativa entre:
- precio_usd y sup_m2_total
- precio_usd y ambientes_cat_code (en reemplazo de ambiente_cat que es categorica)
- precio_usd y lat 
variables que utilizaremos para las predicciones

In [ ]:
#llamamos nuevamente a la funcion describe despues de realizar la eliminacion de outliers 
# para observar los estadisticos media, desv estandar y los minimos y maximos
data.describe()

Conclusiones de la correlacion  de variables predictoras y target:

- Consideraremos como principal variable predictora Superficie total y variable objetivo precio usd por su alta correlación, y su distribución. 
- Evaluaremos el impacto de las variables de ubicación: barrios como dummies, cantidad de ambientes, precio por metro cuadrado, longitud y latitud.


## Implementacion de modelos de regresion lineal

### 1. Regresión lineal simple:

Analizaremos la relacion existente entre la **variable objetivo precio total en dolares (precio_ud)** y su principal **feature la superficie total (sup_m2_total)**

In [ ]:
#preparamos la matriz de features y target
X = data[['sup_m2_total']]
y = data['precio_usd']

#Dvidimos el dataset en train y test
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42)

# Instanciamos el modelo.
lm = linear_model.LinearRegression()

# entrenamos
lm.fit(Xtrain, ytrain)

# Predecimos etiquetas para datos desconocidos.
y_pred = lm.predict(Xtest)
y_pred_train = lm.predict(Xtrain)

In [ ]:
# Imprimimos el intercepto y los coeficientes 
print ('Intercepto =', ' ', lm.intercept_.round(1))
print ('b_sup_m2_total=', ' ', lm.coef_.round(1))

# imprimimos la métrica que mide la bondad de ajusto del modelo. 
print("..........................")
print("R2_train: ", lm.score(Xtrain, ytrain).round(4)) 
print ('R2_test=','', metrics.r2_score(ytest, y_pred).round(4))
print("..........................")
# imprimimos las funciones de perdia de los datos de testeo
print ('MSE_test:', int(metrics.mean_squared_error(ytest, y_pred)))
print ('MSE_train:', int(metrics.mean_squared_error(ytrain, y_pred_train)))
print("..........................")
print ('RMSE_test:', int(np.sqrt(metrics.mean_squared_error(ytest, y_pred))))
print ('RMSE_train:', int(np.sqrt(metrics.mean_squared_error(ytrain, y_pred_train))))
print("..........................")
print ('MAE_test:', int(metrics.mean_absolute_error(ytest, y_pred)))
print ('MAE_train:', int(metrics.mean_absolute_error(ytrain, y_pred_train)))


In [ ]:
# Graficamos la variable X contra la variable Y
plt.figure(figsize=(7,5))
plt.scatter(X, y, s=30, c='black', marker='+', zorder=10)
plt.scatter(X, y)
plt.xlabel("sup_m2_total")
plt.ylabel("Valores reales precio_usd")
plt.title('Relación entre superficie total y precio en dolares')
plt.show()

# Graficamos el modelo
plt.figure(figsize=(7,5))
plt.plot(y,y, '-.',c='grey')
plt.scatter(y_pred, ytest, s=30, c='r', marker='+', zorder=10)
plt.xlabel("Predicciones de precio_usd usando superficie total")
plt.ylabel("Valores reales precio_usd")
plt.title('Comparación entre el modelo y los valores reales de precio_usd')
plt.show()

Resumen de la regresion lineal simple:

Con un R2 en test de 0,6753 observamos que existe una alta correlacion entre la variable precio en dolares y superficie total, es decir, que el aprox. el 67% de los datos de precio usd es predicha por la variable superficie m2 total. El valor de R2:0,6668 en train es ligeramente menor que en test.  

Los graficos corroboran el valor de R2 obtenido en test, en el primer gráfico se observa que hay alta correlación entre ellas, puesto que el precio en dolares aumenta cuando aumenta la superficie total. El segundo gráfico es la comparación entre el valor real vs el valor predicho por el modelo del cual se obtiene el intercepto y la pendiente. La pendiente nos indica que por cada metro cuadrado el precio de la propiedad aumenta en 3480 dolares. 

En cuanto a los errores, fueron calculados MSE, RMSE y MAE. Consideramos que el error que mejor explica los valores predichos por el modelo es MAE ya que es menos sensible a valores atípicos (penaliza mucho más las desviaciones grandes): en este caso los datos tienen cierta variabilidad en los precios cuando aumenta la superficie total por encima de los 300 m2 y hay algunos datos atípicos para superficies menores a 100 m2(grafico 1). 

El error MAE de test es de 66280. Las prediciones del precio_usd se desvian en promedio 66280 dolares del valor real. Tomando en cuenta que la media de los datos totales es de 233841 consideramos que el modelo tiene un buen desempeño y que la relacion establecida de las variables es buena. 

### 2. Regresión lineal múltiple

### Analizaremos el performance del modelo al agregar más variables predictoras:

- variable objetivo: precio en dolares (precio_usd) 
- variables predictoras: superficie total (sup_m2_total), latitud (lat) y longitud (lon), dummies de barrios, dummies de ambientes

Antes, creamos dummies de barrios y ambientes

Descripcion de las dummies a crear:
- Barrios de capital federal: Balvanera, Barrio Norte, Belgrano, Caballito, Flores, Nuñez, Palermo, Recoleta, San Telmo, Villa Crespo, Villa Urquiza

- Ambientes: 1 ambiente, 2 ambientes, 3 ambientes y 4 ambientes o más

In [ ]:
dummies_municipio =pd.get_dummies(data.municipio, prefix='barrio')
# imprimimos 5 filas cualquieras
#dummies_municipio.sample(n=5, random_state=1)


In [ ]:
dummies_municipio.drop(dummies_municipio.columns[0], axis=1, inplace=True)
dummies_municipio.sample(n=5, random_state=1)

In [ ]:
dummies_ambiente =pd.get_dummies(data.ambientes_cat, prefix='ambiente')

In [ ]:
dummies_ambiente.drop(dummies_ambiente.columns[0], axis=1, inplace=True)

# imprimimos 5 filas cualquieras
#dummies_ambiente.sample(n=5, random_state=1)

In [ ]:
data = pd.concat([data, dummies_municipio, dummies_ambiente ], axis=1)

Total de Columnas predictoras a modelar: 16

En orden: 

"sup_m2_total", "lat", "lon", "barrio_Barrio Norte", "barrio_Belgrano", 
"barrio_Caballito", 

"barrio_Flores", "barrio_Nuñez", "barrio_Palermo","barrio_Recoleta", "barrio_San Telmo", "barrio_Villa Crespo","barrio_Villa Urquiza", 

"ambiente_2", "ambiente_3", "ambiente_4 o mas"

In [ ]:
# creamos la variable con las 16 columnas predictoras
features_colums_1 = ["sup_m2_total", "lat", "lon", "barrio_Barrio Norte", "barrio_Belgrano", "barrio_Caballito", "barrio_Flores", "barrio_Nuñez", "barrio_Palermo","barrio_Recoleta", "barrio_San Telmo", "barrio_Villa Crespo","barrio_Villa Urquiza", "ambiente_2", "ambiente_3", "ambiente_4 o mas"]

**Modelo con Ordinary Least Squares regression (OLS)** - sklearn

In [ ]:
# Asignamos las variables predictoras
X_ols = data[features_colums_1]
y_ols = data['precio_usd']

# Dividimos en train y test
X_train, X_test, y_train, y_test = train_test_split(X_ols, y_ols, test_size=0.2, random_state=42)

# Normalizamos los datos
scaler = StandardScaler()
X_train_ss = scaler.fit_transform(X_train)
X_test_ss = scaler.transform(X_test)

  # Instanciamos el modelo
lr= linear_model.LinearRegression()

# Entrenamos el modelo
lr.fit(X_train_ss, y_train)

In [ ]:
# Imprimimos el intercepto y los coeficientes 
#print('Coeficientes: ', lr.coef_)
## para observarlo mejor miramos el nombre con el coeficiente
print (list(zip(features_colums_1, lr.coef_.round(2))))

print(".................................................................................")
print('Intercepto: ', lr.intercept_)
print(".................................................................................")
# imprimimos la métrica que mide la bondad de ajusto del modelo. 
print('R2_train:', r2_score(y_train, lr.predict(X_train_ss)).round(4), '/ R2_test:', r2_score(y_test, lr.predict(X_test_ss)).round(4))
#print('R2_test: ', r2_score(y_test, lr.predict(X_test_ss)))
print(".................................................................................")
# imprimimos las funciones de perdia de los datos de testeo
print('MSE_train:', int(metrics.mean_squared_error(y_train, lr.predict(X_train_ss))), 
'/ MSE_test:', int(metrics.mean_squared_error(y_test, lr.predict(X_test_ss))))
#print('MSE_test:', metrics.mean_squared_error(y_test, lr.predict(X_test_ss)))

print('RMSE_train:', int(np.sqrt(metrics.mean_squared_error(y_train, lr.predict(X_train_ss)))),
'/ RMSE_test:', int(np.sqrt(metrics.mean_squared_error(y_test, lr.predict(X_test_ss)))))

#print('RMSE_test:', int(np.sqrt(metrics.mean_squared_error(y_test, lr.predict(X_test_ss)))))

print('MAE_train:', int(metrics.mean_absolute_error(y_train, lr.predict(X_train_ss))),
'/ MAE_test:', int(metrics.mean_absolute_error(y_test, lr.predict(X_test_ss))))

#print('MAE:', int(metrics.mean_absolute_error(y_test, lr.predict(X_test_ss))))

Interpretacion del modelo de regresion multiple con OLS:

Al considerar 15  nuevas variables predictoras (son 16 con la que usamos en regresion simple) el valor de R2 aumenta de 0,67 a 0,70 en test y se mantiene ligerament similar en train, hay una mejora del performance en la regresion multiple al predecir nuevos datos. 

El error MAE del modelo multivariado es menor cuando predice nuevas observaciones que cuando predice las observaciones con las que fue entrenado. Esto significa que el modelo es útil y que no estamos pudiera no haber overfitting. 

Sobre los coeficientes estimados, se observa que el valor en dolares por unidad de superficie total aumenta considerablemente de 3481 usd a 205815. 
El valor en dolares tiene un incremento mayor con el cambio de latitud que de longitud, y es muy bajo cuando se evalua Barrio Norte 
El precio en dolares tiene una correlacion negativa para los barrios San Telmo y con los ambientes. Pudiera indicar que estas variables no son buenas en la prediccion del precio en dolares o que cuando están evaluadas en conjuntos no contribuyen en la estimación del precio de los inmuebles.
Lo evaluaremos con los p-values en statsmodels

**Modelo  con Statsmodels**

Evaluamos mediante los p values que variables predictoras tienen un buen performance sobre la estimacion del precio en dolares

In [ ]:
# Asignamos las variables predictoras igualmente usando las columnas seleccionadas en regresion OLS sklearn
X_sm = data[features_colums_1] 
y_sm = data[['precio_usd']]

# Normalizamos los datos
scale = StandardScaler()
scale.fit(X_sm)
X_sm = scale.transform(X_sm)

# Modelamos con statsmodels
X_csm = sm.add_constant(X_sm)
model = sm.OLS(y, X_csm).fit()
print(model.summary())

In [ ]:
#vemos el orden en que están presentadas las variables en el resumen de sm
print(data[features_colums_1].columns)

Comparando la regresión multiple sencilla con la regresión de Stastmodel (Sm) podemos observar que Sm permite validar la significancia de los datos obtenidos. 

EL valor de R2 estimado con statsmodel es de de aprox. 0,69, no difiere significativamente de OLS y comparandolo con la regresion sencilla (0,67) aumentó en un 2% la explicacion del precio por las variables predictoras. 

Vemos con los p values (para un valor de significancia de 0,05) de las variables barrio Norte, Belgrano, Nuñez, San telmo, y Villa Urquiza tienen valores de p values mayores al nivel de significancia, lo que indica que estas variables no son buenas predictoras del precio cuando se usan con todas las demás variables.

La no significancia de los barrios mencionados podria estar relacionada con los datos, puede que representen barrios que no sigan la misma tendencia que el resto, pudiendo tener propiedades especificas muy elevadas que se le alejan de la tendencia general. 

Al usar este modelo las variables ambientes son significativas, su p value es menor que el valor de significancia, es decir, pueden considerarse variables predictoras.

Apliquemos regularizacion Ridge y Lasso para comparar los resultados obtenidos con Sm

## Regularización Ridge y Lasso

Ridge

In [ ]:
# Probamos con regularización ridge
lm_ridge = linear_model.RidgeCV(alphas=np.logspace(-10, 3, 200))

model_ridge = lm_ridge.fit(X_train_ss, y_train)

print ('Modelo Ridge:')
print('hiperparametro alpha: ', model_ridge.alpha_)
print('coeficientes ajustados: ', model_ridge.coef_)
print('intercepto: ', model_ridge.intercept_)
print('R2 train: ', model_ridge.score(X_train_ss, y_train))
print('R2 test: ', model_ridge.score(X_test_ss, y_test))

Este método consigue minimizar la influencia de los predictores menos relacionados con la variable respuesta sin llegar a cero: 

Vemos que los coeficientes de los Barrios: Barrio Norte, Belgrano, Nuñez, Recoleta y San Telmo se ven disminuidos al igual que mostraba Sm

Existe mayor valor de prediccion de la los departamentos con 4 ambientes que con 3 y 2 ambientes en el precio en dolares

Lasso

In [ ]:
# Probamos con regularización lasso (tiende a llevar a cero los no explicativos)

lm_lasso = linear_model.LassoCV(alphas=np.logspace(-10, 3, 200), cv=5, tol=0.01)

model_lasso = lm_lasso.fit(X_train_ss, y_train)

print ('Modelo Lasso:')
print('hiperparametro alpha : ', model_lasso.alpha_)
print('coeficientes ajustados: ', model_lasso.coef_)
print('intercepto: ', model_lasso.intercept_)
print('R2 train: ', model_lasso.score(X_train_ss, y_train))
print('R2 test: ', model_lasso.score(X_test_ss, y_test))


Lasso tiene el efecto de forzar a que los coeficientes de los predictores tiendan a cero cuando no influye en el modelo. 

En este caso ninguno de los predictores es cero, sin embargo vemos fuertemente disminuida la variable Barrio Norte y tambien disminuyen los barrios Belgrano, Nuñez, Recoleta y San Telmo. 

Existe mayor valor de prediccion de la los departamentos con 4 ambientes que con 3 y 2 ambientes en el precio en dolares

Ambos modelos de regularización tienen R2 de aprox 0,70 igual que el modelo de regresión lineal multiple, entendiendo que este ultimo no tenia overfitting por corregir.

### Statsmodel sin las variables que dieron un p-value mayor que el valor de significancia (0,05): barrio Norte, Belgrano, Nuñez, San telmo, y Villa Urquiza

Features = sup_m2_total, lat, lon, barrio_Caballito, barrio_Flores, barrio_Palermo,barrio_Recoleta,barrio_Villa Crespo, ambiente_2, ambiente_3", ambiente_4 o mas

In [ ]:
# creamos la variable 
features_colums_2 = ["sup_m2_total", "lat", "lon", "barrio_Caballito", "barrio_Flores", "barrio_Palermo","barrio_Recoleta","barrio_Villa Crespo", "ambiente_2", "ambiente_3", "ambiente_4 o mas"]

X_sm = data[features_colums_2] #usando las 7 variables predictoraas
y_sm = data[['precio_usd']]

# Normalizamos los datos
scale = StandardScaler()
scale.fit(X_sm)
X_sm = scale.transform(X_sm)

# Modelamos con statsmodels
X_csm = sm.add_constant(X_sm)
model = sm.OLS(y, X_csm).fit()
print(model.summary())

Al eliminar los barrios que presentaban baja prediccion del precio en dolares vemos que el modelo mantiene el R2 de aprox. 0,70.

Es decir, que con el uso de estas variables el modelo es capaz de predecir el 70% de los valores de precio en dolares.

En este caso que no detectamos overfitting en el modelo el uso de Statsmodel fue de mayor ayuda que regularización. 